In [60]:
import sys
from pathlib import Path

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import src.reload; src.reload.reload()

In [6]:
nft_ops = []
art_house_ops = []
money_trs = []
trs_info = []

print('Filtering NFT, ArtHouse and money ops...')

for tr in src.tr.iter.iter_tr():
    trs_info.append([
        tr['hash'],
        src.utils.iso_date_to_stamp(tr['time']),
        tr['ops'][0]['row_id'],
        tr['ops'][-1]['row_id'],
        tr['fpath'].name,
    ])

    tr_volume = 0
    for op in tr['ops']:
        op_hash = op['hash']

        if op['sender'] == src.config.name2addr['nft_contract']:
            raise Exception(f'Not expecting nft_contract to send messages: {op_hash}')

        if op.get('receiver') == src.config.name2addr['nft_contract']:
            nft_ops.append(op)

        if op.get('receiver') == src.config.name2addr['art_house_contract']:
            art_house_ops.append(op)

        if op['status'] != 'applied':
            continue

        assert op['volume'] >= 0
        tr_volume += op['volume']

    if tr_volume > 0:
        money_trs.append(tr)


  0%|                                                                                                                                                           | 0/26 [00:00<?, ?it/s]

Filtering NFT, ArtHouse and money ops...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:16<00:00,  1.55it/s]

In [61]:
from collections import Counter

trs = money_trs


# def parse_trs(trs):
if True:
    money_state = src.contracts.money_state.MoneyState()
    money_state = src.contracts.state_utils.StateRecorder(money_state)

    for tr in trs:

        nft_calls = Counter()
        last_nft_op = None
        for op in tr['ops']:
            if op['type'] == 'transaction' and op['receiver'] == src.config.name2addr['nft_contract']:
                nft_calls[op['parameters']['call']] += 1
                last_nft_op = op
        nft_calls = dict(nft_calls)

        money_delta = src.tr.utils.get_tr_money_delta(tr)
        known_money_delta = {
            src.config.addr2name[addr]: delta
            for addr, delta in money_delta.items()
            if addr in src.config.addr2name
        }

        if nft_calls == {'transfer': 1}:
            
            tr_ops = tr['ops']
            assert len(tr_ops) in [6, 7]
            
            if len(tr['ops']) == 7:
                assert tr['ops'][0]['type'] == 'reveal'
                tr_ops = tr['ops'][1:]
            
            assert len(tr_ops) == 6

            # get buyer money
            assert tr_ops[0]['parameters']['call'] == 'collect'
            assert tr_ops[0]['receiver'] == src.config.name2addr['art_house_contract']
            assert tr_ops[0]['volume'] > 0
            payer = tr_ops[0]['sender']
            price = tr_ops[0]['volume']

            # pay royalties
            assert 'parameters' not in tr_ops[1]
            assert tr_ops[1]['volume'] > 0
            royalties_receiver = tr_ops[1]['receiver']
            royalties = tr_ops[1]['volume']

            # pay comission
            assert 'parameters' not in tr_ops[2]
            assert tr_ops[2]['receiver'] == src.config.name2addr['comission_wallet']
            assert tr_ops[2]['volume'] > 0
            comission = tr_ops[2]['volume']

            # pay to seller
            assert 'parameters' not in tr_ops[3]
            assert tr_ops[3]['volume'] > 0
            seller = tr_ops[3]['receiver']
            seller_income = tr_ops[3]['volume']

            # give hDAO tokens
            assert tr_ops[4]['receiver'] == src.config.name2addr['hdao_contract']
            assert tr_ops[4]['parameters']['call'] == 'hDAO_batch'
            assert tr_ops[4]['volume'] == 0

            # transfer NFT
            assert tr_ops[5]['parameters']['call'] == 'transfer'
            assert tr_ops[5]['receiver'] == src.config.name2addr['nft_contract']
            assert tr_ops[5]['volume'] == 0

            transfer = tr_ops[5]['parameters']['value']['transfer']
            assert len(transfer) == 1
            sender = transfer[0]['from_']

            # when the first transfer between users (with non-zero XTZ volume) will happen,
            # this assert will fail
            assert sender == src.config.name2addr['art_house_contract']

            assert len(transfer[0]['txs']) == 1
            receiver = transfer[0]['txs'][0]['to_']

            token_id = int(transfer[0]['txs'][0]['token_id'])
            token_count = int(transfer[0]['txs'][0]['amount'])

            money_state.apply_swap_sell(
                row_id=last_nft_op['row_id'],
                token_id=token_id,
                token_count=token_count,
                payer=payer,
                price=price,
                seller=seller,
                seller_income=seller_income,
                comission=comission,
                royalties_receiver=royalties_receiver,
                royalties=royalties,
            )
            continue

        assert nft_calls == {}
        known_money_addrs = set(known_money_delta.keys())

        assert 'comission_wallet' in known_money_addrs

        if 'baking_benjamins' in known_money_addrs:
            assert known_money_addrs == {'baking_benjamins', 'comission_wallet'}
            assert known_money_delta['baking_benjamins'] < 0
            assert known_money_delta['comission_wallet'] > 0
            assert known_money_delta['comission_wallet'] < -known_money_delta['baking_benjamins']
            money_state.apply_comission_wallet_baking_percent(
                row_id=tr['ops'][0]['row_id'],
                sender=src.config.name2addr['baking_benjamins'],
                volume=known_money_delta['comission_wallet'],
            )
            continue

        assert known_money_addrs == {'comission_wallet'}
        assert known_money_delta['comission_wallet'] < 0
        other_addrs = set(money_delta.keys()) - {src.config.name2addr['comission_wallet']}
        assert len(other_addrs) == 1
        other_addr = list(other_addrs)[0]
        volume = money_delta[other_addr]
        assert volume > 0
        assert money_delta == {
            src.config.name2addr['comission_wallet']: -volume,
            other_addr: volume
        }
        money_state.apply_comission_wallet_spending(
            row_id=tr['ops'][0]['row_id'],
            receiver=other_addr,
            volume=volume,
        )

# money_state.log

In [ ]:
assert last_nft_op['parameters']['call'] == 'transfer'
transfer = last_nft_op['parameters']['value']['transfer']
assert len(transfer) == 1
sender = transfer[0]['from_']

# when the first transfer between users (with non-zero price) will happen,
# this assert will fail
assert sender == src.config.name2addr['art_house_contract']
assert money_delta[src.config.name2addr['art_house_contract']] < 1e-10

assert len(transfer[0]['txs']) == 1
receiver = transfer[0]['txs'][0]['to_']

token_id = int(transfer[0]['txs'][0]['token_id'])
token_count = int(transfer[0]['txs'][0]['amount'])
comission_wallet_income = money_delta[src.config.name2addr['comission_wallet']]

beneficiaries = {}
payers = {}
for addr, delta in money_delta.items():
    if addr in [
        src.config.name2addr['art_house_contract'],
        src.config.name2addr['comission_wallet'],
    ]:
        continue
    if delta > 0:
        beneficiaries[addr] = delta
    elif delta < 0:
        payers[addr] = delta

royalties = 0
royalties_receiver = ''

if receiver == src.config.name2addr['comission_wallet']:
    assert len(payers) == 0
    assert len(beneficiaries) == 1
    beneficiary = list(beneficiaries)[0]
    payer = src.config.name2addr['comission_wallet']
    assert comission_wallet_income < 0
    price = abs(comission_wallet_income) / (1.0 - 0.025)
    comission = price * 0.025

elif len(beneficiaries) == 0:
    assert len(payers) == 1
    payer = list(payers)[0]
    assert money_delta[payer] == -comission_wallet_income
    assert comission_wallet_income > 0
    price = abs(comission_wallet_income)
    beneficiary = src.config.name2addr['comission_wallet']
    comission = price * 0.025

elif len(beneficiaries) == 2:
    assert len(payers) == 1
    assert comission_wallet_income > 0
    comission = comission_wallet_income
    payer = list(payers)[0]
    price = -money_delta[payer]
    royalties_receiver, beneficiary = list(beneficiaries)
    royalties = money_delta[royalties_receiver]

else:
    assert len(payers) == 1
    assert len(beneficiaries) == 1, tr['hash']
    assert comission_wallet_income > 0
    comission = comission_wallet_income
    payer = list(payers)[0]
    price = -money_delta[payer]
    beneficiary = list(beneficiaries)[0]


assert type(payer) is str
assert type(beneficiary) is str
assert price > 0
assert comission > 0
assert abs(comission - price * 0.025) < 1e-6, (comission, price, royalties, tr['hash'])
